In [1]:
import pandas as pd
import os
import datetime
import arcpy


C:\Users\htran\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Functions for unit calculations (Before intersect with fishnet) => Get densities
def add_and_calculate_geodesic_unit(fc):
    #Create the temporary id for joining fields later
    arcpy.AddField_management(fc,'tempid','LONG')
    arcpy.CalculateField_management(fc,"tempid",'!OBJECTID!','PYTHON')
    #Calculate the area(land+water) by GEODESIC method => Kytt recommended
    arcpy.management.CalculateGeometryAttributes(fc, "GEODESIC_AREA AREA_GEODESIC", "", "SQUARE_METERS")
def get_and_calculate_water(fc, waterfc, state):
    #Create water mask to calculate the water area
    water_mask = arcpy.analysis.Clip(fc, waterfc, f"in_memory/{state}water_clipped")
    arcpy.management.CalculateGeometryAttributes(water_mask, "GEODESIC_WATER AREA_GEODESIC", "", "SQUARE_METERS")
    #join the GEODESIC_WATER field back to the boundary feature based on tempid
    arcpy.management.JoinField(fc, "tempid", water_mask, "tempid", ["GEODESIC_WATER"])
def calculate_land_area(fc):
    #Add field GEODESIC_LAND and calculate by AREA - WATER => LAND
    arcpy.AddField_management(fc, "GEODESIC_LAND", "FLOAT")
    with arcpy.da.UpdateCursor(fc, ["GEODESIC_AREA", "GEODESIC_WATER", "GEODESIC_LAND"]) as cursor:
            for row in cursor:
                if row[1] == None:
                    row[1] = 0 #After join there are Null values so I make it to be 0
                    row[2] = row[0] #GEODESIC_LAND is being calculated
                    cursor.updateRow(row)
                else:
                    row[2] = row[0] - row[1] 
                    cursor.updateRow(row)
def calculate_density(fc):
    #Get all the population related fields
    fields = [field.name for field in arcpy.ListFields(fc)]
    new_fields = fields[35:92] + fields[93:-6]
    #Calculate the densities for all those fields
    for variableField in new_fields:
        arcpy.AddField_management(fc,f"{variableField}_DENS",'DOUBLE')
        exp = f"get_dens(!{variableField}!, !GEODESIC_LAND!)"

        codeblock = """def get_dens(field, land):
                            if (land == 0 and field >= 0):
                                return 0
                            else:
                                return field/land

        """

        arcpy.management.CalculateField(fc,f"{variableField}_DENS",exp,"PYTHON", codeblock)
        
#Functions for PIXEL calculations (After intersect with fishnet) => Get counts
def add_and_calculate_geodesic_pixel(intersectFc):
    arcpy.management.AddField(intersectFc, "INTERSECTID", "LONG")
    arcpy.management.CalculateField(intersectFc, "INTERSECTID", "!OBJECTID!", "PYTHON")
    arcpy.management.CalculateGeometryAttributes(intersectFc, "PIXEL_AREA AREA_GEODESIC", "", "SQUARE_METERS")
def get_and_calculate_water_pixel(intersectFc, waterfc, state):
    #Create water mask to calculate the water area
    water_mask = arcpy.analysis.Clip(intersectFc, waterfc, f"in_memory/{state}pixel_water_clipped")
    arcpy.management.CalculateGeometryAttributes(water_mask, "PIXEL_WATER AREA_GEODESIC", "", "SQUARE_METERS")
    #join the PIXEL_WATER field back to the boundary feature based on tempid
    arcpy.management.JoinField(intersectFc, "INTERSECTID", water_mask, "INTERSECTID", ["PIXEL_WATER"])
def calculate_land_pixel(intersectFc):
    #Add field PIXEL_LAND and calculate by AREA - WATER => LAND
    arcpy.AddField_management(intersectFc, "PIXEL_LAND", "FLOAT")
    with arcpy.da.UpdateCursor(intersectFc, ["PIXEL_AREA", "PIXEL_WATER", "PIXEL_LAND"]) as cursor:
            for row in cursor:
                if row[1] == None:
                    row[1] = 0 #After join there are Null values so I make it to be 0
                    cursor.updateRow(row)
                row[2] = row[0] - row[1] #GEODESIC_LAND is being calculated
                cursor.updateRow(row)
def calculate_count(intersectFc):
    #Get all the population related fields
    dens_fields = [field.name for field in arcpy.ListFields(intersectFc) if "_DENS" in field.name]
    #Calculate the densities for all those fields
    for variableField in dens_fields:
        arcpy.AddField_management(intersectFc,f"{variableField[:-5]}_CNT",'DOUBLE')
        exp = f"get_count(!{variableField}!, !PIXEL_LAND!)"

        codeblock = """def get_count(field, land):
                            if (land == 0 and field >= 0):
                                return 0
                            else:
                                return field*land

        """

        arcpy.management.CalculateField(intersectFc,f"{variableField[:-5]}_CNT",exp,"PYTHON", codeblock)
        
#Function for getting the COUNT for the fields based on the intersected pixels 
def get_stats(fishnet, intersectFc, state):
    # create list of fields to generate statistics for
    statsFields = [["PIXEL_AREA","SUM"],["PIXEL_WATER","SUM"],["PIXEL_LAND","SUM"]]
    joinFields = ["SUM_PIXEL_AREA","SUM_PIXEL_WATER","SUM_PIXEL_LAND"]
    cntFields = arcpy.ListFields(intersectFc,"*_CNT")
    if len(cntFields)>0:
        [statsFields.append([field.name,"SUM"]) for field in cntFields]
        [joinFields.append("SUM_"+f.name) for f in cntFields]
    sumTable = f"in_memory/{state}summarizeTable1"
#     fishnetInMem = arcpy.management.CopyFeatures(fishnet, "in_memory/fishnet")
    # summarize the variables for each grid cell
    arcpy.Statistics_analysis(intersectFc,sumTable,statsFields,"PIXELID")
    arcpy.JoinField_management(fishnetInMem,'PIXELID',sumTable,'PIXELID',joinFields)
    for joinField in joinFields:
        arcpy.AlterField_management(fishnetInMem,joinField,joinField.replace("SUM_",""),joinField.replace("SUM_",""))
def get_raster(fishnet, state):
    # Convert fishnet to grids
    cellSize = 0.0083333333/2 #it's used for 1km but now 500m => cellSize/2
    gridFields = ["PIXEL_LAND","PIXEL_WATER"] + [field.name for field in arcpy.ListFields(fishnet, "*_CNT")]
    raster_folder = os.mkdir(os.path.join(r"G:/usgrids2020/raster/", state))
    outFolder = f"G:/usgrids2020/raster/{state}"
    for gridField in gridFields:
        outGrid = outFolder + os.sep + state + "_" + gridField + ".tif"
        arcpy.PolygonToRaster_conversion(fishnet,gridField,outGrid,'CELL_CENTER','#',cellSize)

In [ ]:
gdb_path = "G:/usgrids2020/final_gdbs"
gdb_list = os.listdir(gdb_path)
gdb_list.sort()
for state in gdb_list[3:]:
    print(f"Processing for {state[3:5].upper()}")
    readTime = datetime.datetime.now()
    arcpy.env.workspace = os.path.join(gdb_path, state)
    #Import the feature for boundary and water
    fc = f"usa{state[3:5]}_joined"
    fcInMem = arcpy.management.CopyFeatures(fc, f"in_memory/usa{state[3:5]}_fcInMem")
    water_fc = f"G:/usgrids2020/water_state/{state[3:5]}_water_layer.shp"
    waterInMem = arcpy.management.CopyFeatures(water_fc, f"in_memory/usa{state[3:5]}_waterInMem")
    add_and_calculate_geodesic_unit(fcInMem)
    get_and_calculate_water(fcInMem, waterInMem, state[3:5])
    calculate_land_area(fcInMem)
    calculate_density(fcInMem)
    print("Starting intersect fishnet to boundary...")
    fish_net = f"G:/usgrids2020/fishnets/usa{state[3:5]}_fishnet.gdb/usa{state[3:5]}_fishnet"
    fishnetInMem = arcpy.management.CopyFeatures(fish_net, f"in_memory/{state[3:5]}_fishnetInMem")
    intersectOut = f"in_memory/{state[3:5]}intersect_fc"
    arcpy.analysis.Intersect([fcInMem,fish_net], intersectOut, "NO_FID")
    add_and_calculate_geodesic_pixel(intersectOut)
    get_and_calculate_water_pixel(intersectOut, waterInMem, state[3:5])
    calculate_land_pixel(intersectOut)
    calculate_count(intersectOut)
    print("Summarizing...")
    get_stats(fishnetInMem, intersectOut, state[3:5])
    get_raster(fishnetInMem, state[3:5].upper())
    print(f"Finished processing for {state[3:5].upper()} in {str(datetime.datetime.now() - readTime)}")

Processing for AZ
Starting intersect fishnet to boundary...
Summarizing...
Finished processing for AZ in 13:10:25.897409
Processing for CA
Starting intersect fishnet to boundary...


In [7]:
 gdb_list[2][3:5]

'ar'

In [7]:
os.path.basename(fc)

'usadc_joined'

In [6]:
for field in arcpy.ListFields(fishnetInMem):
    print(field.name)
# with arcpy.da.SearchCursor(fish_net, )

OBJECTID
Shape
Id
gridcode
VALUETEXT
Shape_Length
Shape_Area
PIXELID
PIXEL_AREA
PIXEL_WATER
PIXEL_LAND
ATOTPOPMT_CNT
ATOTPOPFT_CNT
ATOTPOPBT_CNT
A000_004MT_CNT
A005_009MT_CNT
A010_014MT_CNT
A015_019MT_CNT
A020_024MT_CNT
A025_029MT_CNT
A030_034MT_CNT
A035_039MT_CNT
A040_044MT_CNT
A045_049MT_CNT
A050_054MT_CNT
A055_059MT_CNT
A060_064MT_CNT
A065_069MT_CNT
A070_074MT_CNT
A075_079MT_CNT
A080_084MT_CNT
A085plusMT_CNT
A000_004FT_CNT
A005_009FT_CNT
A010_014FT_CNT
A015_019FT_CNT
A020_024FT_CNT
A025_029FT_CNT
A030_034FT_CNT
A035_039FT_CNT
A040_044FT_CNT
A045_049FT_CNT
A050_054FT_CNT
A055_059FT_CNT
A060_064FT_CNT
A065_069FT_CNT
A070_074FT_CNT
A075_079FT_CNT
A080_084FT_CNT
A085plusFT_CNT
A000_004BT_CNT
A005_009BT_CNT
A010_014BT_CNT
A015_019BT_CNT
A020_024BT_CNT
A025_029BT_CNT
A030_034BT_CNT
A035_039BT_CNT
A040_044BT_CNT
A045_049BT_CNT
A050_054BT_CNT
A055_059BT_CNT
A060_064BT_CNT
A065_069BT_CNT
A070_074BT_CNT
A075_079BT_CNT
A080_084BT_CNT
A085plusBT_CNT
TOTALPOP_CNT
AUND1_CNT
A1TO4_CNT
A5TO17_CNT
A

In [74]:
# create list of fields to generate statistics for
statsFields = [["PIXEL_AREA","SUM"],["PIXEL_WATER","SUM"],["PIXEL_LAND","SUM"]]
joinFields = ["SUM_PIXEL_AREA","SUM_PIXEL_WATER","SUM_PIXEL_LAND"]
cntFields = arcpy.ListFields(intersectOut,"*_CNT")
if len(cntFields)>0:
    [statsFields.append([field.name,"SUM"]) for field in cntFields]
#     [joinFields.append("SUM_"+f.name) for f in cntFields]
sumTable = "in_memory/summarizeTable2"
fishnetInMem = arcpy.management.CopyFeatures(fish_net, "in_memory/fishnet2")


In [75]:
# summarize the variables for each grid cell
arcpy.Statistics_analysis(intersectOut,sumTable,statsFields,"PIXELID")

arcpy.JoinField_management(fishnetInMem,'PIXELID',sumTable,'PIXELID',joinFields)
for joinField in joinFields:
    arcpy.AlterField_management(fishnetInMem,joinField,joinField.replace("SUM_",""),joinField.replace("SUM_",""))

<Result 'in_memory\\summarizeTable2'>

In [76]:
for field in arcpy.ListFields(sumTable):
    print(field.name)

OBJECTID
PIXELID
FREQUENCY
SUM_PIXEL_AREA
SUM_PIXEL_WATER
SUM_PIXEL_LAND
SUM_ATOTPOPMT_CNT
SUM_ATOTPOPFT_CNT
SUM_ATOTPOPBT_CNT
SUM_A000_004MT_CNT
SUM_A005_009MT_CNT
SUM_A010_014MT_CNT
SUM_A015_019MT_CNT
SUM_A020_024MT_CNT
SUM_A025_029MT_CNT
SUM_A030_034MT_CNT
SUM_A035_039MT_CNT
SUM_A040_044MT_CNT
SUM_A045_049MT_CNT
SUM_A050_054MT_CNT
SUM_A055_059MT_CNT
SUM_A060_064MT_CNT
SUM_A065_069MT_CNT
SUM_A070_074MT_CNT
SUM_A075_079MT_CNT
SUM_A080_084MT_CNT
SUM_A085plusMT_CNT
SUM_A000_004FT_CNT
SUM_A005_009FT_CNT
SUM_A010_014FT_CNT
SUM_A015_019FT_CNT
SUM_A020_024FT_CNT
SUM_A025_029FT_CNT
SUM_A030_034FT_CNT
SUM_A035_039FT_CNT
SUM_A040_044FT_CNT
SUM_A045_049FT_CNT
SUM_A050_054FT_CNT
SUM_A055_059FT_CNT
SUM_A060_064FT_CNT
SUM_A065_069FT_CNT
SUM_A070_074FT_CNT
SUM_A075_079FT_CNT
SUM_A080_084FT_CNT
SUM_A085plusFT_CNT
SUM_A000_004BT_CNT
SUM_A005_009BT_CNT
SUM_A010_014BT_CNT
SUM_A015_019BT_CNT
SUM_A020_024BT_CNT
SUM_A025_029BT_CNT
SUM_A030_034BT_CNT
SUM_A035_039BT_CNT
SUM_A040_044BT_CNT
SUM_A045_049BT_CNT

In [70]:
joinFields

['SUM_PIXEL_AREA',
 'SUM_PIXEL_WATER',
 'SUM_PIXEL_LAND',
 'SUM_ATOTPOPMT_CNT',
 'SUM_ATOTPOPFT_CNT',
 'SUM_ATOTPOPBT_CNT',
 'SUM_A000_004MT_CNT',
 'SUM_A005_009MT_CNT',
 'SUM_A010_014MT_CNT',
 'SUM_A015_019MT_CNT',
 'SUM_A020_024MT_CNT',
 'SUM_A025_029MT_CNT',
 'SUM_A030_034MT_CNT',
 'SUM_A035_039MT_CNT',
 'SUM_A040_044MT_CNT',
 'SUM_A045_049MT_CNT',
 'SUM_A050_054MT_CNT',
 'SUM_A055_059MT_CNT',
 'SUM_A060_064MT_CNT',
 'SUM_A065_069MT_CNT',
 'SUM_A070_074MT_CNT',
 'SUM_A075_079MT_CNT',
 'SUM_A080_084MT_CNT',
 'SUM_A085plusMT_CNT',
 'SUM_A000_004FT_CNT',
 'SUM_A005_009FT_CNT',
 'SUM_A010_014FT_CNT',
 'SUM_A015_019FT_CNT',
 'SUM_A020_024FT_CNT',
 'SUM_A025_029FT_CNT',
 'SUM_A030_034FT_CNT',
 'SUM_A035_039FT_CNT',
 'SUM_A040_044FT_CNT',
 'SUM_A045_049FT_CNT',
 'SUM_A050_054FT_CNT',
 'SUM_A055_059FT_CNT',
 'SUM_A060_064FT_CNT',
 'SUM_A065_069FT_CNT',
 'SUM_A070_074FT_CNT',
 'SUM_A075_079FT_CNT',
 'SUM_A080_084FT_CNT',
 'SUM_A085plusFT_CNT',
 'SUM_A000_004BT_CNT',
 'SUM_A005_009BT_CNT',
 '

In [71]:
with arcpy.da.SearchCursor(sumTable, [joinFields, "PIXELID"]) as cursor:
    for i in range(10):
        for row in cursor:
            print(row[3:])
            break

(0.00292720253153697, 0.0032199227846906675, 0.006147125316227638, 0.0, 0.0, 0.00029272025315369706, 0.00029272025315369706, 0.0, 0.0, 0.0, 0.0, 0.00019514683543579803, 9.757341771789902e-05, 0.0, 0.0, 0.0, 0.00039029367087159607, 9.757341771789902e-05, 0.0, 0.0007805873417431921, 0.0007805873417431921, 0.0, 0.00029272025315369706, 0.0, 0.00019514683543579803, 0.0, 9.757341771789902e-05, 0.00029272025315369706, 0.000683013924025293, 0.00019514683543579803, 0.00019514683543579803, 0.00019514683543579803, 0.00029272025315369706, 0.0005854405063073941, 0.0, 0.0, 0.0, 0.00019514683543579803, 0.0, 0.0, 0.00029272025315369706, 0.00029272025315369706, 0.00048786708858949507, 0.0, 9.757341771789902e-05, 0.00029272025315369706, 0.000683013924025293, 0.00039029367087159607, 0.00029272025315369706, 0.00019514683543579803, 0.00029272025315369706, 0.0005854405063073941, 0.00039029367087159607, 9.757341771789902e-05, 0.0, 0.0009757341771789901, 0.0007805873417431921, 0.006147125316227638, 0.0, 0.0, 

In [66]:
cnt_fields = [field.name for field in arcpy.ListFields(sumTable) if "_CNT" in field.name]
with arcpy.da.SearchCursor(sumTable, [cnt_fields, "PIXELID"]) as cursor:
    for i in range(10):
        for row in cursor:
            if row[-1] == -559847290:
                print(row)
                break

(0.00292720253153697, 0.0032199227846906675, 0.006147125316227638, 0.0, 0.0, 0.00029272025315369706, 0.00029272025315369706, 0.0, 0.0, 0.0, 0.0, 0.00019514683543579803, 9.757341771789902e-05, 0.0, 0.0, 0.0, 0.00039029367087159607, 9.757341771789902e-05, 0.0, 0.0007805873417431921, 0.0007805873417431921, 0.0, 0.00029272025315369706, 0.0, 0.00019514683543579803, 0.0, 9.757341771789902e-05, 0.00029272025315369706, 0.000683013924025293, 0.00019514683543579803, 0.00019514683543579803, 0.00019514683543579803, 0.00029272025315369706, 0.0005854405063073941, 0.0, 0.0, 0.0, 0.00019514683543579803, 0.0, 0.0, 0.00029272025315369706, 0.00029272025315369706, 0.00048786708858949507, 0.0, 9.757341771789902e-05, 0.00029272025315369706, 0.000683013924025293, 0.00039029367087159607, 0.00029272025315369706, 0.00019514683543579803, 0.00029272025315369706, 0.0005854405063073941, 0.00039029367087159607, 9.757341771789902e-05, 0.0, 0.0009757341771789901, 0.0007805873417431921, 0.006147125316227638, 0.0, 0.0, 

In [10]:
dens_fields = [field.name for field in arcpy.ListFields(intersectOut) if "_DENS" in field.name]
    #Calculate the densities for all those fields
for variableField in dens_fields:
    arcpy.AddField_management(intersectOut,f"{variableField[:-5]}_CNT",'DOUBLE')

In [4]:
for field in arcpy.ListFields(intersectOut):
    print(field.name)

OBJECTID
Shape
STATEFP20
COUNTYFP20
TRACTCE20
BLOCKCE20
GEOID20
NAME20
MTFCC20
UR20
UACE20
UATYPE20
FUNCSTAT20
ALAND20
AWATER20
INTPTLAT20
INTPTLON20
HOUSING20
POP20
USCID
ISO
UCADMIN0
NAME0
UCADMIN1
NAME1
UCADMIN2
NAME2
UCADMIN3
NAME3
UCADMIN4
NAME4
UCADMIN5
NAME5
CENSUS_YEA
POP_CONTEX
ATOTPOPMT
ATOTPOPFT
ATOTPOPBT
A000_004MT
A005_009MT
A010_014MT
A015_019MT
A020_024MT
A025_029MT
A030_034MT
A035_039MT
A040_044MT
A045_049MT
A050_054MT
A055_059MT
A060_064MT
A065_069MT
A070_074MT
A075_079MT
A080_084MT
A085plusMT
A000_004FT
A005_009FT
A010_014FT
A015_019FT
A020_024FT
A025_029FT
A030_034FT
A035_039FT
A040_044FT
A045_049FT
A050_054FT
A055_059FT
A060_064FT
A065_069FT
A070_074FT
A075_079FT
A080_084FT
A085plusFT
A000_004BT
A005_009BT
A010_014BT
A015_019BT
A020_024BT
A025_029BT
A030_034BT
A035_039BT
A040_044BT
A045_049BT
A050_054BT
A055_059BT
A060_064BT
A065_069BT
A070_074BT
A075_079BT
A080_084BT
A085plusBT
union_uid
TOTALPOP
AUND1
A1TO4
A5TO17
A18TO24
AUND25
A25OV
A25TO64
A65TO79
A80OV
WOMCHIL

In [12]:
dens_fields = [field.name for field in arcpy.ListFields(fcInMem) if "_DENS" in field.name]
dens_fields

['ATOTPOPMT_DENS',
 'ATOTPOPFT_DENS',
 'ATOTPOPBT_DENS',
 'A000_004MT_DENS',
 'A005_009MT_DENS',
 'A010_014MT_DENS',
 'A015_019MT_DENS',
 'A020_024MT_DENS',
 'A025_029MT_DENS',
 'A030_034MT_DENS',
 'A035_039MT_DENS',
 'A040_044MT_DENS',
 'A045_049MT_DENS',
 'A050_054MT_DENS',
 'A055_059MT_DENS',
 'A060_064MT_DENS',
 'A065_069MT_DENS',
 'A070_074MT_DENS',
 'A075_079MT_DENS',
 'A080_084MT_DENS',
 'A085plusMT_DENS',
 'A000_004FT_DENS',
 'A005_009FT_DENS',
 'A010_014FT_DENS',
 'A015_019FT_DENS',
 'A020_024FT_DENS',
 'A025_029FT_DENS',
 'A030_034FT_DENS',
 'A035_039FT_DENS',
 'A040_044FT_DENS',
 'A045_049FT_DENS',
 'A050_054FT_DENS',
 'A055_059FT_DENS',
 'A060_064FT_DENS',
 'A065_069FT_DENS',
 'A070_074FT_DENS',
 'A075_079FT_DENS',
 'A080_084FT_DENS',
 'A085plusFT_DENS',
 'A000_004BT_DENS',
 'A005_009BT_DENS',
 'A010_014BT_DENS',
 'A015_019BT_DENS',
 'A020_024BT_DENS',
 'A025_029BT_DENS',
 'A030_034BT_DENS',
 'A035_039BT_DENS',
 'A040_044BT_DENS',
 'A045_049BT_DENS',
 'A050_054BT_DENS',
 'A

In [ ]:
gdb_path = "G:/usgrids2020/final_gdbs"
gdb_list = os.listdir(gdb_path)
gdb_list.sort()
for state in gdb_list[44:]:
    print(f"Processing for {state[3:5].upper()}")
    readTime = datetime.datetime.now()
    arcpy.env.workspace = os.path.join(gdb_path, state)
    #Import the feature for boundary and water
    fc = f"usa{state[3:5]}_joined"
    fcInMem = arcpy.management.CopyFeatures(fc, f"in_memory/usa{state[3:5]}_InMem")
    water_fc = f"G:/usgirds2020/water_state/{state[3:5]}_water_layer.shp"
    waterInMem = arcpy.management.CopyFeatures(water_fc, f"in_memory/usa{state[3:5]}_InMem")
    #Get shape area by m^2 and add the field into the attributes
    readTime = datetime.datetime.now()
    #Create the temporary id for joining fields later
    def add_and_calculate_geodesic(fc):
        arcpy.AddField_management(fcInMem,'tempid','LONG')
        arcpy.CalculateField_management(fcInMem,"tempid",'!OBJECTID!','PYTHON')
        #Calculate the area(land+water) by GEODESIC method => Kytt recommended
        arcpy.management.CalculateGeometryAttributes(fcInMem, "GEODESIC_AREA AREA_GEODESIC", "", "SQUARE_METERS")
    def get_and_calculate_water(fc, waterfc):
        #Create water mask to calculate the water area
        water_mask = arcpy.analysis.Clip(fcInMem, waterInMem, "in_memory/clipped7")
        arcpy.management.CalculateGeometryAttributes(water_mask, "GEODESIC_WATER AREA_GEODESIC", "", "SQUARE_METERS")
        #join the GEODESIC_WATER field back to the boundary feature based on tempid
        arcpy.management.JoinField(fcInMem, "tempid", water_mask, "tempid", ["GEODESIC_WATER"])
    def calculate_land_area(fc):
        #Add field GEODESIC_LAND and calculate by AREA - WATER => LAND
        arcpy.AddField_management(fcInMem, "GEODESIC_LAND", "FLOAT")
        with arcpy.da.UpdateCursor(fcInMem, ["GEODESIC_AREA", "GEODESIC_WATER", "GEODESIC_LAND", "tempid" ]) as cursor:
            for i in range(10):
                for row in cursor:
                    if row[1] == None:
                        row[1] = 0 #After join there are Null values so I make it to be 0
                        cursor.updateRow(row)
                    row[2] = row[0] - row[1] #GEODESIC_LAND is being calculated
                    cursor.updateRow(row)
    def calculate_density(fc):
        #Get all the population related fields
        fields = [field.name for field in arcpy.ListFields(fcInMem)]
        new_fields = fields[35:92] + fields[93:-6]
        #Calculate the densities for all those fields
        for variableField in new_fields:
            arcpy.AddField_management(fcInMem,variableField+"_DENS",'DOUBLE')
            exp = f"get_dens(!{variableField}!, !GEODESIC_LAND!)"

            codeblock = """def get_dens(field, land):
                                if (land == 0 and field >= 0):
                                    return 0
                                else:
                                    return field/land

            """

            arcpy.management.CalculateField(testfc,f"{variableField}_DENS",exp,"PYTHON", codeblock)
#     proj = arcpy.SpatialReference(54009)
#     projfcInMem = f"in_memory/{state[3:5]}projfcInMem"
#     arcpy.management.Project(fcInMem, projfcInMem,proj)
#     arcpy.AddField_management(projfcInMem,'PROJECTED_AREA','DOUBLE')
#     arcpy.AddField_management(projfcInMem, "Per_diff", "FLOAT")
#     arcpy.CalculateField_management(projfcInMem,"PROJECTED_AREA",'!shape.area@SQUAREMETERS!','PYTHON')
#     with arcpy.da.UpdateCursor(projfcInMem, ["PROJECTED_AREA", "ALAND20", "AWATER20", "Per_diff"]) as cursor:
#         for i in range(1000):
#             for row in cursor:
#                 row[3] = ((row[0] - (row[1]+row[2]))/(row[1] + row[2])*100)
#                 cursor.updateRow(row)
#     drop_fields = [field.name for field in arcpy.ListFields(projfcInMem)]
#     drop_fields = drop_fields[13:15] + drop_fields[-3:]
#     arcpy.management.DeleteField(projfcInMem, drop_fields, "KEEP_FIELDS")
    arcpy.management.CopyFeatures(fcInMem, f"G:/usgrids2020/final_gdbs/usa{state[3:5]}.gdb/usa{state[3:5]}_area")
    print("The script finished in: " + str(datetime.datetime.now() - readTime))

In [ ]:

with arcpy.da.SearchCursor(fcInMem, ["TOTALPOP","HHUNIT","OCC","TOTALPOP_DENS","HHUNIT_DENS","OCC_DENS"]) as cursor:
    for row in cursor:
        print(row)

In [ ]:
fields = [field.name for field in arcpy.ListFields(fcInMem)]
new_fields = fields[35:92] + fields[93:-6]
new_fields

In [ ]:
testfc = arcpy.management.CopyFeatures(fcInMem, "in_memory/test23")

for variableField in new_fields:
    arcpy.AddField_management(testfc,variableField+"_DENS",'DOUBLE')
    exp = f"get_dens(!{variableField}!, !GEODESIC_LAND!)"
    
    codeblock = """def get_dens(field, land):
                        if (land == 0 and field >= 0):
                            return 0
                        else:
                            return field/land
                            
    """
    
    arcpy.management.CalculateField(testfc,f"{variableField}_DENS",exp,"PYTHON", codeblock)

In [ ]:
dens_fields

In [ ]:

with arcpy.da.SearchCursor(testfc, ["TOTALPOP","HHUNIT","OCC","TOTALPOP_DENS","HHUNIT_DENS","OCC_DENS"]) as cursor:
    for row in cursor:
        print(row)

In [ ]:
#Get shape area by m^2 and add the field into the attributes
arcpy.env.workspace = "G:/usgrids2020/final_gdbs/usade.gdb"
fc = "usade_joined"
fcInMem = arcpy.management.CopyFeatures(fc, "in_memory/fcInMem10")
water_fc = r"G:/usgrids2020/water_state/de_water_layer.shp"
waterInMem = arcpy.management.CopyFeatures(water_fc, "in_memory/waterInMem10")
arcpy.AddField_management(fcInMem,'tempid','LONG')
arcpy.AddField_management(fcInMem, "GEODESIC_LAND", "FLOAT")
arcpy.CalculateField_management(fcInMem,"tempid",'!OBJECTID!','PYTHON')
arcpy.management.CalculateGeometryAttributes(fcInMem, "GEODESIC_AREA AREA_GEODESIC", "", "SQUARE_METERS")
water_mask = arcpy.analysis.Clip(fcInMem, waterInMem, "in_memory/clipped10")
arcpy.management.CalculateGeometryAttributes(water_mask, "GEODESIC_WATER AREA_GEODESIC", "", "SQUARE_METERS")
arcpy.management.JoinField(fcInMem, "tempid", water_mask, "tempid", ["GEODESIC_WATER"])
with arcpy.da.UpdateCursor(fcInMem, ["GEODESIC_AREA", "GEODESIC_WATER", "GEODESIC_LAND", "tempid" ]) as cursor:
    for i in range(10):
        for row in cursor:
            if row[1] == None:
                row[1] = 0
                cursor.updateRow(row)
            row[2] = row[0] - row[1]
            cursor.updateRow(row)
# arcpy.management.AddField("GEODESIC_WATER", "DOUBLE")
# with arcpy.da.UpdateCursor(water_mask, ["GEODESIC_AREA", "GEODESIC_WATER", "ALAND20", "AWATER20", ]) as cursor:
#     for i in range(1000):
#             for row in cursor:
#                 print("The difference: " + str((row[0] + row[1]) - (row[2] + row[3])))
#                 break


In [ ]:
print(arcpy.management.GetCount(fcInMem))


In [ ]:
with arcpy.da.UpdateCursor(fcInMem, ["GEODESIC_AREA", "GEODESIC_WATER", "tempid" ]) as cursor:
    for i in range(10):
            for row in cursor:
                if row[1] == None:
                    row[1] = 0
                    cursor.updateRow(row)
#                 break

In [ ]:
with arcpy.da.SearchCursor(fcInMem, ["GEODESIC_AREA", "GEODESIC_WATER", "GEODESIC_LAND", "tempid" ]) as cursor:
    for i in range(10):
            for row in cursor:
                print(row)
#                 break

In [ ]:
# arcpy.management.CalculateGeometryAttributes(fcInMem, "GEODESIC_AREA AREA_GEODESIC", "", "SQUARE_METERS")

water_mask = r"G:/usgrids2020/final_gdbs/usade.gdb/clipped"
watermaskInMem = arcpy.management.CopyFeatures(water_mask, "in_memory/watermaskInMem")
arcpy.management.CalculateGeometryAttributes(watermaskInMem, "GEODESIC_WATER AREA_GEODESIC", "", "SQUARE_METERS")

In [ ]:
with arcpy.da.SearchCursor(watermaskInMem, ["GEODESIC_WATER", "ALAND20", "AWATER20"]) as cursor:
    for i in range(100):
        for row in cursor:
            print(row)
            break

In [ ]:
difference = pd.Series(x)
difference.plot()
plt.title("Percentage difference from Geodesic method")

In [ ]:
proj = arcpy.SpatialReference(54009)
projfcInMem = "in_memory/projfcInMem8"
arcpy.management.Project(fcInMem, projfcInMem,proj)
arcpy.AddField_management(projfcInMem,'PROJECTED_AREA','DOUBLE')
arcpy.AddField_management(projfcInMem, "Per_diff", "FLOAT")
arcpy.CalculateField_management(projfcInMem,"PROJECTED_AREA",'!shape.area@SQUAREMETERS!','PYTHON')
x = list()
with arcpy.da.UpdateCursor(projfcInMem, ["PROJECTED_AREA", "ALAND20", "AWATER20", "Per_diff"]) as cursor:
    for i in range(1000):
        for row in cursor:
#             row[3] = ((row[0] - (row[1]+row[2]))/(row[1] + row[2])*100)
            print(row[0], row[1], row[2])
            row[3] = ((row[0] - (row[1]+row[2]))/(row[1] + row[2])*100)
            cursor.updateRow(row)
#             break


In [ ]:
difference = pd.Series(x)
difference.plot(kind = "line")
plt.title("Percentage difference from Projection method")

In [ ]:
drop_fields = [field.name for field in arcpy.ListFields(projfcInMem)]
drop_fields = drop_fields[13:15] + drop_fields[-3:]
arcpy.management.DeleteField(projfcInMem, drop_fields, "KEEP_FIELDS")


In [ ]:
arcpy.management.CopyFeatures(projfcInMem, f"G:/usgrids2020/final_gdbs/usadc.gdb/usadc_area")

In [ ]:
for field in arcpy.ListFields(projfcInMem):
    print(field.name)